## Install python packages

In [ ]:
!pip install numpy keras matplotlib pydot python-dotenv tensorflow

  Using cached nvidia_cublas_cu12-12.3.4.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.3.101-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cuda_nvcc_cu12-12.3.107-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.3.107-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.3.101-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cudnn_cu12-8.9.7.29-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cufft_cu12-11.0.12.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.4.107-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.5.4.101-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.2.0.103-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyli

-----

## Load environment variables

In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

-----

## Install graphviz (for linux)

In [ ]:
USE_SUDO = os.getenv('USE_SUDO')

bash_command = "apt-get install graphviz -y"

if USE_SUDO:
    SUDO_PASSWORD = os.getenv('SUDO_PASSWORD')
    
    bash_command = "echo {sudo_password} | sudo -S {command}".format(sudo_password = SUDO_PASSWORD, command = bash_command)
    
!{bash_command}

-----

## Import libraries

In [ ]:
%matplotlib inline

from os import path
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import InputLayer, BatchNormalization, Conv2DTranspose, Dense, Reshape, ReLU, LeakyReLU, Conv2D, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import plot_model

-----

## Verificar el uso de GPU

In [ ]:
print("Numbers of GPUs availables: {gpus_number}".format(gpus_number = len(tf.config.list_physical_devices("GPU"))))

-----

## Define constants

In [ ]:
# Directories used for the jupyter notebook
DIRECTORY_PATHS = {}

# Create the directory paths
DIRECTORY_PATHS["models"] = path.join('.', 'models')
DIRECTORY_PATHS["models_architecture"] = path.join(DIRECTORY_PATHS["models"], 'architecture')
DIRECTORY_PATHS["models_results"] = path.join(DIRECTORY_PATHS["models"], 'results')

MODEL_ARCHITECTURE_DIRECTORY = DIRECTORY_PATHS["models_architecture"]
MODEL_RESULTS_DIRECTORY = DIRECTORY_PATHS["models_results"]

# Name of the used models in this jupyter notebook
MODEL_NAMES = {
    "generator": "Generator",
    "discriminator": "Discriminator",
    "dcgan": "DCGAN"
}

# Extension of the architecture files (used by plot_model)
EXTENSION = "pdf"

# Use to map the numeric label to categorical label
CATEGORICAL_LABELS = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

# Constants used to show images
NUM_ROWS = 5
NUM_COLUMNS = 5
PADDING = .2

# Constans used to get random sample
SAMPLE_SIZE = NUM_ROWS * NUM_COLUMNS

# Constants used to train the DCGAN
NUM_EPOCHS = 32
BATCH_SIZE = 64

# BACTH_SIZE = NUM_REAL_DATA_SAMPLE + NUM_FAKE_DATA_SAMPLE
NUM_REAL_DATA_SAMPLE = int(BATCH_SIZE / 2)
NUM_FAKE_DATA_SAMPLE = NUM_REAL_DATA_SAMPLE

-----

## Create directories

In [ ]:
for directory in DIRECTORY_PATHS.values():
    if not path.isdir(directory):
        os.mkdir(directory)

-----

## Define functions

In [ ]:
def normalize_image(unnormalized_image):
    return (unnormalized_image / 127.5) - 1.0

def unnormalize_image(normalized_image):
    return (127.5 * (1.0 + normalized_image)).astype(int)
    
def get_model_architecture_file(base_path, file_name, extension):
    full_file_name = "{file_name}.{extension}".format(file_name = file_name, extension = extension)
    return path.join(base_path, full_file_name)

def get_random_sample(images, labels, sample_size):
    len_images = len(images)
    len_labels = len(labels)
    if len_images == len_labels:
        sample_indices = np.random.permutation(len_images)
        sample_indices = sample_indices[:sample_size]
        return images[sample_indices], labels[sample_indices]

def get_categorical_labels(labels, categorical_labels):
    labels = labels.copy().flatten()
    if isinstance(categorical_labels, dict):
        return [categorical_labels[label] for label in labels]
    
def get_images(images, labels=None, num_rows=1, num_columns=1, padding=.2, title = None):
    fig, ax = plt.subplots(num_rows, num_columns)
    fig.tight_layout(pad=padding)
    
    subplots = ax.flatten() if isinstance(ax, np.ndarray) else [ax]
    num_subplots = len(subplots)

    if title:
        fig.suptitle(title, y = 1.1)
    
    for i in range(num_subplots):
        subplot = subplots[i]
        image = images[i]
        
        subplot.axis("off")
        subplot.imshow(images[i])
        
        if labels:
            label = labels[i]
            subplot.set_title(labels[i])
    
    return fig

-----

## Datasets

### Load dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

### A. Train data

#### Obtain data sample

In [ ]:
sample_x_train, sample_y_train = get_random_sample(x_train, y_train, SAMPLE_SIZE)

#### Obtain categorical labels

In [ ]:
sample_categorical_y_train = get_categorical_labels(sample_y_train, CATEGORICAL_LABELS)

#### Show images

In [ ]:
get_images(sample_x_train, sample_categorical_y_train, num_rows=NUM_ROWS, num_columns = NUM_COLUMNS, padding = PADDING, title="Images from train dataset");

### B. Test data

#### Obtain data sample

In [ ]:
sample_x_test, sample_y_test = get_random_sample(x_test, y_test, SAMPLE_SIZE)

#### Obtain categorical labels

In [ ]:
sample_categorical_y_test = get_categorical_labels(sample_y_test, CATEGORICAL_LABELS)

#### Show images

In [ ]:
get_images(sample_x_test, sample_categorical_y_test, num_rows = NUM_ROWS, num_columns = NUM_COLUMNS, padding = PADDING, title="Images from test dataset");

-----

## Generator model

In [ ]:
def build_generator(input_shape):
    generator = Sequential()

    generator.add(InputLayer(shape = input_shape))
    generator.add(Dense(4 * 4 * 1024))
    generator.add(ReLU())
    generator.add(BatchNormalization())
    generator.add(Reshape((4, 4, 1024)))

    generator.add(Conv2DTranspose(256, kernel_size = 3, strides = 2, padding = "same"))
    generator.add(ReLU())
    generator.add(BatchNormalization())

    generator.add(Conv2DTranspose(128, kernel_size = 3, strides = 2, padding = "same"))
    generator.add(ReLU())
    generator.add(BatchNormalization())

    generator.add(Conv2DTranspose(3, kernel_size = 3, strides = 2, padding = "same", activation="tanh"))

    generator.compile(run_eagerly = True)
    
    generator.summary()

    return generator

### Obtain the path to save the architecture file

In [ ]:
GENERATOR_MODEL_ARCHITECTURE_FILE = get_model_architecture_file(MODEL_ARCHITECTURE_DIRECTORY, MODEL_NAMES["generator"], EXTENSION)

### Build generator

In [ ]:
generator_input_shape = (100,)

generator = build_generator(generator_input_shape)

### Save the plot model

In [ ]:
plot_model(generator, to_file = GENERATOR_MODEL_ARCHITECTURE_FILE, show_shapes = True, show_layer_names = True)

### Create random input to generate images

In [ ]:
random_input = np.random.rand(NUM_ROWS * NUM_COLUMNS, 100)

### Print shape

In [ ]:
print(random_input.shape)

### Use the generator to create new images

In [ ]:
result = [unnormalize_image(normalized_image) for normalized_image in generator.predict(random_input)]

### Show the images

In [ ]:
get_images(result, num_rows = NUM_ROWS, num_columns = NUM_COLUMNS, padding = PADDING, title = "Images generated by generator");

-----

## Discriminator model

In [ ]:
def build_discriminator(input_shape):
    discriminator = Sequential()

    discriminator.add(Conv2D(128, input_shape = input_shape, kernel_size = 3, strides = 2, padding = "same"))
    discriminator.add(LeakyReLU())
    discriminator.add(BatchNormalization())

    discriminator.add(Conv2D(256, kernel_size = 3, strides = 2, padding = "same"))
    discriminator.add(LeakyReLU())
    discriminator.add(BatchNormalization())

    discriminator.add(Conv2D(1024, kernel_size = 3, strides = 2, padding = "same"))
    discriminator.add(LeakyReLU())
    discriminator.add(BatchNormalization())
    
    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation = "sigmoid"))

    optimizer = Adam(learning_rate = 0.01)

    discriminator.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"], run_eagerly = True)

    discriminator.summary()

    return discriminator

### Obtain the path to save the architecture file

In [20]:
DISCRIMINATOR_MODEL_ARCHITECTURE_FILE = get_model_architecture_file(MODEL_ARCHITECTURE_DIRECTORY, MODEL_NAMES["discriminator"], EXTENSION)

### Build discriminator

In [ ]:
discriminator_input_shape = (32, 32, 3, )

discriminator = build_discriminator(discriminator_input_shape)

### Save the plot model

In [ ]:
plot_model(discriminator, to_file = DISCRIMINATOR_MODEL_ARCHITECTURE_FILE, show_shapes = True, show_layer_names = True)

-----

## DCGAN model

In [ ]:
def build_dcgan(generator, discriminator, input_shape = None):
    # The discriminator's weights are not trainnable when the dcgan is trained.
    discriminator.trainnable = False

    generator.name = "Generator"
    discriminator.name = "Discriminator"
    
    dcgan = Sequential()

    dcgan.add(InputLayer(shape = input_shape))  
    dcgan.add(generator)
    dcgan.add(discriminator)

    optimizer = Adam(learning_rate = 0.01)

    dcgan.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"], run_eagerly = True)

    dcgan.summary()
    
    return dcgan

### Obtain the path to save the architecture file

In [ ]:
DCGAN_MODEL_ARCHITECTURE_FILE = get_model_architecture_file(MODEL_ARCHITECTURE_DIRECTORY, MODEL_NAMES["dcgan"], EXTENSION)

### Build DCGAN

In [ ]:
dcgan = build_dcgan(generator, discriminator, input_shape = generator_input_shape)

### Save the plot model

In [ ]:
plot_model(dcgan, to_file = DCGAN_MODEL_ARCHITECTURE_FILE, show_shapes = True, show_layer_names = True)

-----

## Train DCGAN

### Define functions used to train dcgan

In [32]:
def get_real_image_batch(images, batch_size = 32):
    images = images.copy()

    # Shuffle the images
    np.random.shuffle(images)

    # Obtain the number of complete batches
    num_batch = int(len(images) / batch_size)
    # Obtain the number of total images used to generate complete batches
    num_images = num_batch * batch_size

    # Obtain the images 
    images = images[ : num_images]

    # Preprocess the image
    images = np.array(list(map(normalize_image, images)))

    for i in range(num_batch):
        yield images[i * batch_size : (i + 1) * batch_size]

def get_discriminator_sample(fake_data, fake_labels, real_data, real_labels):
    train_data = np.concatenate([fake_data, real_data], axis = 0)
    train_labels = np.concatenate([fake_labels, real_labels], axis = 0)

    return train_data, train_labels

### Train DCGAN

In [ ]:
for epoch in range(1, NUM_EPOCHS + 1): 
    for real_images in get_real_image_batch(x_train, batch_size = NUM_REAL_DATA_SAMPLE):
        # ----- Train discriminator -----
        
        # Generate noise by the generator to create a new images
        noise = np.random.rand(NUM_FAKE_DATA_SAMPLE, 100)

        # Generate the fake images and fake labels
        fake_images = generator.predict_on_batch(noise).astype(np.float32)
        fake_labels = np.zeros((NUM_FAKE_DATA_SAMPLE, 1), dtype = np.float32)
        
        # Generate the real labels
        real_labels = np.ones((NUM_REAL_DATA_SAMPLE, 1)).astype(np.float32)
        
        # Combine the fake and real images and fake and real labels
        discriminator_train_data, discriminator_train_labels = get_discriminator_sample(fake_images, fake_labels, real_images, real_labels)
        
        # First train the discriminator
        discriminator.train_on_batch(discriminator_train_data, discriminator_train_labels)
        
        # ----- Train the dcgan -----
    
        # Generate new noise
        dcgan_train_data = np.random.rand(BATCH_SIZE, 100).astype(np.float32)
        
        # The label is changed to deceive to discriminator, equivalent to maximize log(D(G(z))). See https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
        dcgan_train_labels = np.ones((BATCH_SIZE, 1)).astype(np.float32)
    
        # Train dcgan
        dcgan.train_on_batch(dcgan_train_data, dcgan_train_labels)

    # ----- Show results -----
        
    # Generate new noise
    noise = np.random.rand(NUM_ROWS * NUM_COLUMNS, 100)
    
    new_images = np.array([unnormalize_image(normalized_image) for normalized_image in generator.predict_on_batch(noise)])
    
    # Save new images
    filename = path.join(MODEL_RESULTS_DIRECTORY, "result_in_epoch{num_epoch}.{extension}".format(num_epoch = epoch, extension = EXTENSION))
    
    plot_title = "Images generated by generator in the epochs: {epoch}".format(epoch = epoch)
    plot_images = get_images(new_images, num_rows = NUM_ROWS, num_columns = NUM_COLUMNS, padding = PADDING, title = plot_title)

    plot_images.savefig(filename)